In [3]:
from util import format_query , query_to_df
import pandas as pd
import numpy as np

In [4]:
## Try collection data for pde articles

max_results=3e3
cat='math.AP'
query = format_query(cat=cat)
pde = query_to_df(query=query,max_results=max_results)


In [26]:
pde.msc_tags.sample(10)

## 2556, 2059, 1445

1445    [42A38 (Primary), 42C30, 34B24 (Secondary)]
2325                                           None
2622                                 [35L75, 35B25]
226                    [37K40, 37K60, 34C15, 34A33]
1354                                           None
1292                                           None
1663                                           None
2140                                           None
684                                            None
2467                                 [35A21, 35D99]
Name: msc_tags, dtype: object

In [34]:
## Find the msc subject tags within the categories
import regex

def find_msc(cat_list):
    pattern = r'\b\d{2}[0-9a-zA-Z]{3}\b'
    out = []
    for cat in cat_list:
        tags = regex.findall(pattern,cat)
        for tag in tags:
            out.append(tag)
    if out == []:
        return None
    else:
        return out

        

In [35]:
## Now create a new column for msc tags

pde['msc_tags'] = pde.categories.apply(find_msc)

In [49]:
## Check to see if we are getting expected results

msc = pde.msc_tags.sample(10)
for tag in msc:
    print(tag)


None
None
None
['35J46', '35J50', '35A01']
['35P20', '35Q74', '74J05']
['42B37', '35J25', '35J70']
None
['60H15', '60H10', '37H15', '37H20']
['35J35', '35J48', '35R01', '49Q10', '53A05', '53A10', '53A30', '53C42']
['35B45', '35B10']


In [51]:
## what fraction of these articles has non-zero msc tags?

tagged = pde.msc_tags.count()
fraction = tagged/len(pde)
print(fraction)



0.4746666666666667


In [1]:
## Now we are going to see if we can extract the MSC codes using xml parsing with beautifulsoup

from bs4 import BeautifulSoup
import requests 

url = r'https://cran.r-project.org/web/classifications/MSC.html'

source = requests.get(url)


In [2]:
source.headers['content-type']

'text/html'

In [4]:
source.encoding

'ISO-8859-1'

In [7]:
document = source.text

In [8]:
soup = BeautifulSoup(document, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   MSC Classification Codes
  </title>
  <link href="../CRAN_web.css" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <h2>
   MSC Classification Codes
  </h2>
  <ul>
   <li id="code:00-xx">
    <a href="http://www.ams.org/msc/msc.html?t=00-xx">
     00-xx
    </a>
    : General
    <ul>
     <li id="code:00-01">
      00-01: Instructional exposition (textbooks, tutorial papers, etc.)
     </li>
     <li id="code:00-02">
      00-02: Research exposition (monographs, survey articles)
     </li>
     <li id="code:00Axx">
      <a href="http://www.ams.org/msc/msc.html?t=00Axx">
       00Axx
      </a>
      : General and miscellaneous specific topics
      <ul>
       <li id="code:00A05">
        00A05: General mathematics
       </li>
       <li id="code:00A06">
        00A06: Mathematics for nonmathem

In [9]:
## Another way--read the msc tags from pdf

!pip install PyPDF2

                                              0.0/232.6 kB ? eta -:--:--
     -----------------------------------   225.3/232.6 kB 13.4 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 2.9 MB/s eta 0:00:00


In [21]:
import PyPDF2
import regex

with open('msc2020.pdf', 'rb') as file:

    reader = PyPDF2.PdfReader(file)
    print(len(reader.pages))
    page = reader.pages[0]
    raw_text = page.extract_text()

    lines = raw_text.split('\n')

    subject_dict = {}
    for line in lines[2:]:
        subject_dict[str(line[:2])] = line[2:]


subject_dict['44'] = 'Integral transforms, operational calculus'
subject_dict['45'] = 'Integral equations'

for k in subject_dict.keys():
    subject_dict[k] = regex.sub(r'\x0b','ff',subject_dict[k])

print(subject_dict)

224
{'00': 'General and overarching topics; collections', '01': 'History and biography', '03': 'Mathematical logic and foundations', '05': 'Combinatorics', '06': 'Order, lattices, ordered algebraic structures', '08': 'General algebraic systems', '11': 'Number theory', '12': 'Field theory and polynomials', '13': 'Commutative algebra', '14': 'Algebraic geometry', '15': 'Linear and multilinear algebra; matrix theory', '16': 'Associative rings and algebras', '17': 'Nonassociative rings and algebras', '18': 'Category theory; homological algebra', '19': 'K-theory', '20': 'Group theory and generalizations', '22': 'Topological groups, Lie groups', '26': 'Real functions', '28': 'Measure and integration', '30': 'Functions of a complex variable', '31': 'Potential theory', '32': 'Several complex variables and analytic spaces', '33': 'Special functions', '34': 'Ordinary differential equations', '35': 'Partial differential equations', '37': 'Dynamical systems and ergodic theory', '39': 'Difference a

In [22]:
subject_dict.pop('1')
print(subject_dict)

{'00': 'General and overarching topics; collections', '01': 'History and biography', '03': 'Mathematical logic and foundations', '05': 'Combinatorics', '06': 'Order, lattices, ordered algebraic structures', '08': 'General algebraic systems', '11': 'Number theory', '12': 'Field theory and polynomials', '13': 'Commutative algebra', '14': 'Algebraic geometry', '15': 'Linear and multilinear algebra; matrix theory', '16': 'Associative rings and algebras', '17': 'Nonassociative rings and algebras', '18': 'Category theory; homological algebra', '19': 'K-theory', '20': 'Group theory and generalizations', '22': 'Topological groups, Lie groups', '26': 'Real functions', '28': 'Measure and integration', '30': 'Functions of a complex variable', '31': 'Potential theory', '32': 'Several complex variables and analytic spaces', '33': 'Special functions', '34': 'Ordinary differential equations', '35': 'Partial differential equations', '37': 'Dynamical systems and ergodic theory', '39': 'Difference and f

In [23]:
## Save this dictionary as a json file

import json

json_subjects = json.dumps(subject_dict)
with open('./data/msc_subjects.json','w+') as file:
    file.write(json_subjects)
    

In [27]:
import util
import importlib
importlib.reload(util)

util.msc_subjects()

{'00': 'General and overarching topics; collections',
 '01': 'History and biography',
 '03': 'Mathematical logic and foundations',
 '05': 'Combinatorics',
 '06': 'Order, lattices, ordered algebraic structures',
 '08': 'General algebraic systems',
 '11': 'Number theory',
 '12': 'Field theory and polynomials',
 '13': 'Commutative algebra',
 '14': 'Algebraic geometry',
 '15': 'Linear and multilinear algebra; matrix theory',
 '16': 'Associative rings and algebras',
 '17': 'Nonassociative rings and algebras',
 '18': 'Category theory; homological algebra',
 '19': 'K-theory',
 '20': 'Group theory and generalizations',
 '22': 'Topological groups, Lie groups',
 '26': 'Real functions',
 '28': 'Measure and integration',
 '30': 'Functions of a complex variable',
 '31': 'Potential theory',
 '32': 'Several complex variables and analytic spaces',
 '33': 'Special functions',
 '34': 'Ordinary differential equations',
 '35': 'Partial differential equations',
 '37': 'Dynamical systems and ergodic theory'

In [33]:
## Next we make a dictionary consisting of all other subject tags

with open('msc2020.pdf', 'rb') as file:

    reader = PyPDF2.PdfReader(file)
    page = reader.pages[3]
    raw_text = page.extract_text()


Some comments on msc tags:

- Most are of the form \d\d[A-Z]\d\d.
- All tags that are XXX99 are 'None of the above, but in this section'.
- Some of the text ends with [...] which lists similar tags in other subjects.
- Some of the text contains (...) which contains more info about content.
- \d\d[A-Z] contains finer topic info than the [\d\d] tag
- some escaped characters are not read in properly.

Maybe we 

In [75]:
## Try splitting on a pattern \d\d[A-Z]xx

pattern = r'\b\d\d[A-Z]xx\b'
splitting = regex.split(pattern,raw_text)
# for line in splitting:
    # print(line + 'END')

print(splitting[1])

## Within each of these, find all text between two instances of the pattern 
tag_pattern = r'(\b\d\d[A-Z]\d\d)\s(.*)(?!\b\d\d[A-Z]\d\d\b)'

patterns = regex.findall(tag_pattern, splitting[1])


 General and miscellaneous specic topics
00A05 Mathematics in general
00A06 Mathematics for nonmathematicians (engineering, social sciences, etc.)
00A07 Problem booksfFor open problems, see 00A27 g
00A08 Recreational mathematics
00A09 Popularization of mathematics
00A15 Bibliographies for mathematics in general [See also 01A70 and the classication number {00 in the other
sections]
00A17 External book reviews
00A20 Dictionaries and other general reference works [See also the classication number {00 in the other sections]
00A22 Formularies
00A27 Lists of open problems
00A30 Philosophy of mathematics [See also 03A05]
00A35 Methodology of mathematics fFor mathematics education, see 97-XX g
00A64 Mathematics and literature
00A65 Mathematics and music
00A66 Mathematics and visual arts
00A67 Mathematics and architecture
00A69 General applied mathematics fFor physics, see 00A79 and Sections 70 through 86 g
00A71 General theory of mathematical modeling
00A72 General theory of simulation
00A7

In [76]:
patterns

[('00A05', 'Mathematics in general'),
 ('00A06',
  'Mathematics for nonmathematicians (engineering, social sciences, etc.)'),
 ('00A07', 'Problem booksfFor open problems, see 00A27 g'),
 ('00A08', 'Recreational mathematics'),
 ('00A09', 'Popularization of mathematics'),
 ('00A15',
  'Bibliographies for mathematics in general [See also 01A70 and the classi\x0ccation number {00 in the other'),
 ('00A17', 'External book reviews'),
 ('00A20',
  'Dictionaries and other general reference works [See also the classi\x0ccation number {00 in the other sections]'),
 ('00A22', 'Formularies'),
 ('00A27', 'Lists of open problems'),
 ('00A30', 'Philosophy of mathematics [See also 03A05]'),
 ('00A35',
  'Methodology of mathematics fFor mathematics education, see 97-XX g'),
 ('00A64', 'Mathematics and literature'),
 ('00A65', 'Mathematics and music'),
 ('00A66', 'Mathematics and visual arts'),
 ('00A67', 'Mathematics and architecture'),
 ('00A69',
  'General applied mathematics fFor physics, see 00A79 

In [80]:
## Turn this into a dict
dict = {}
for item in patterns:
    k , v = item
    dict[k] = v

for item in dict.items():
    print(item)

('00A05', 'Mathematics in general')
('00A06', 'Mathematics for nonmathematicians (engineering, social sciences, etc.)')
('00A07', 'Problem booksfFor open problems, see 00A27 g')
('00A08', 'Recreational mathematics')
('00A09', 'Popularization of mathematics')
('00A15', 'Bibliographies for mathematics in general [See also 01A70 and the classi\x0ccation number {00 in the other')
('00A17', 'External book reviews')
('00A20', 'Dictionaries and other general reference works [See also the classi\x0ccation number {00 in the other sections]')
('00A22', 'Formularies')
('00A27', 'Lists of open problems')
('00A30', 'Philosophy of mathematics [See also 03A05]')
('00A35', 'Methodology of mathematics fFor mathematics education, see 97-XX g')
('00A64', 'Mathematics and literature')
('00A65', 'Mathematics and music')
('00A66', 'Mathematics and visual arts')
('00A67', 'Mathematics and architecture')
('00A69', 'General applied mathematics fFor physics, see 00A79 and Sections 70 through 86 g')
('00A71', 'G

In [81]:

## Within each of these, find all text between two instances of the pattern 
tag_pattern = r'(\b\d\d[A-Z]\d\d)\s(.*)(?!\b\d\d[A-Z]\d\d\b)'
dict = {}

with open('msc2020.pdf', 'rb') as file:

    reader = PyPDF2.PdfReader(file)
    for page in reader.pages:
        page_text = page.extract_text()

        ## Find all the msc tags 

        tags = regex.findall(tag_pattern, page_text)
        
        for item in tags:
            k , v = item
            dict[k] = v


In [82]:
dict

{'00A05': 'Mathematics in general',
 '00A06': 'Mathematics for nonmathematicians (engineering, social sciences, etc.)',
 '00A07': 'Problem booksfFor open problems, see 00A27 g',
 '00A08': 'Recreational mathematics',
 '00A09': 'Popularization of mathematics',
 '00A15': 'Bibliographies for mathematics in general [See also 01A70 and the classi\x0ccation number {00 in the other',
 '00A17': 'External book reviews',
 '00A20': 'Dictionaries and other general reference works [See also the classi\x0ccation number {00 in the other sections]',
 '00A22': 'Formularies',
 '00A27': 'Lists of open problems',
 '00A30': 'Philosophy of mathematics [See also 03A05]',
 '00A35': 'Methodology of mathematics fFor mathematics education, see 97-XX g',
 '00A64': 'Mathematics and literature',
 '00A65': 'Mathematics and music',
 '00A66': 'Mathematics and visual arts',
 '00A67': 'Mathematics and architecture',
 '00A69': 'General applied mathematics fFor physics, see 00A79 and Sections 70 through 86 g',
 '00A71': 'G

Need to fix:

There are random special characters present that seem to replace 2 latin characters

- '\x0c' -> 'fi'
- '\x0b' -> 'ff'
- '\r' -> 'fl'
- '\x13E' -> E
- '\xf7 ' -> umlaut over the next character, so we just delete this. Note the space at the end.
- '\x0e' -> 'ffi'
- '\x15'
- '\x10'
- 'x1f'
- '\x10'
- '\x12 ' -> '' 
- '\x00'
- '\x153' -> 
- '\x19' -> \pi
- '\x1b' -> \sigma
- '\x1e' -> \phi
- '\x0f' -> 'ffl'


In [86]:
def clean_msc_dict(dict):
    for item in dict.items():
        k , v = item 
        v = regex.sub(r'\x0c','fi',v)
        v = regex.sub(r'\x0b','ff',v)
        v = regex.sub(r'\r','fl',v)
        v = regex.sub(r'\xf7 ','',v)
        v = regex.sub(r'\x0e','ffi',v)
        v = regex.sub(r'\x13','',v)
        dict[k] = v
        return dict


In [87]:
clean_dict = clean_msc_dict(dict)

In [88]:
for item in clean_dict.items():
    print(item)
    print()

('00A05', 'Mathematics in general')

('00A06', 'Mathematics for nonmathematicians (engineering, social sciences, etc.)')

('00A07', 'Problem booksfFor open problems, see 00A27 g')

('00A08', 'Recreational mathematics')

('00A09', 'Popularization of mathematics')

('00A15', 'Bibliographies for mathematics in general [See also 01A70 and the classification number {00 in the other')

('00A17', 'External book reviews')

('00A20', 'Dictionaries and other general reference works [See also the classification number {00 in the other sections]')

('00A22', 'Formularies')

('00A27', 'Lists of open problems')

('00A30', 'Philosophy of mathematics [See also 03A05]')

('00A35', 'Methodology of mathematics fFor mathematics education, see 97-XX g')

('00A64', 'Mathematics and literature')

('00A65', 'Mathematics and music')

('00A66', 'Mathematics and visual arts')

('00A67', 'Mathematics and architecture')

('00A69', 'General applied mathematics fFor physics, see 00A79 and Sections 70 through 86 g')


In [173]:
import pandas as pd
codes = pd.read_csv('./data/MSC_2020.csv', encoding='windows-1252', on_bad_lines='skip')

In [156]:
codes.head()

,code\ttext\tdescription
0,"00-XX\t""General and overarching topics; collections""\t""General and overarching topics; collections"""
1,"00Axx\t""General and miscellaneous specific topics""\t""General and miscellaneous specific topics"""
2,"00A05\t""Mathematics in general""\t""Mathematics in general"""
3,"00A08\t""Recreational mathematics""\t""Recreational mathematics"""
4,"00A09\t""Popularization of mathematics""\t""Popularization of mathematics"""


In [174]:
## Look for all entries that start with a valid msc tag pattern

pattern = '\d\d[A-Z]\d\d'

def check_valid(entry):
    if regex.match(pattern,entry):
        return True
    else:
        return False

valid_codes = codes.loc[codes['code\ttext\tdescription'].apply(check_valid)]


In [175]:
valid_codes

,code\ttext\tdescription
2,"00A05\t""Mathematics in general""\t""Mathematics in general"""
3,"00A08\t""Recreational mathematics""\t""Recreational mathematics"""
4,"00A09\t""Popularization of mathematics""\t""Popularization of mathematics"""
5,"00A15\t""Bibliographies for mathematics in general""\t""Bibliographies for mathematics in general [See also 01A70 and the classification number --00 in the other sections]"""
6,"00A17\t""External book reviews""\t""External book reviews"""
...,...
3964,"97P80\t""Artificial intelligence (educational aspects)""\t""Artificial intelligence (educational aspects)"""
3966,"97U10\t""Comprehensive works on educational material and media and educational technology in mathematics education""\t""Comprehensive works on educational material and media and educational technology in mathematics education"""
3967,"97U30\t""Teachers' manuals and planning aids (aspects of mathematics education)""\t""Teachers' manuals and planning aids (aspects of mathematics education)"""
3968,"97U60\t""Manipulative materials (aspects of mathematics education)""\t""Manipulative materials (aspects of mathematics education)"""


In [182]:
dict = {}

for entry in valid_codes['code\ttext\tdescription']:
    split = entry.split('\t')
    code = split[0]
    desc = split[1][1:-1]
    dict[code] = desc


In [183]:
dict

{'00A05': 'Mathematics in general',
 '00A08': 'Recreational mathematics',
 '00A09': 'Popularization of mathematics',
 '00A15': 'Bibliographies for mathematics in general',
 '00A17': 'External book reviews',
 '00A20': 'Dictionaries and other general reference works',
 '00A22': 'Formularies',
 '00A27': 'Lists of open problems',
 '00A30': 'Philosophy of mathematics',
 '00A64': 'Mathematics and literature',
 '00A65': 'Mathematics and music',
 '00A66': 'Mathematics and visual arts',
 '00A67': 'Mathematics and architecture',
 '00A71': 'General theory of mathematical modeling',
 '00A72': 'General theory of simulation',
 '00A79': 'Physics',
 '00B05': 'Collections of abstracts of lectures',
 '00B10': 'Collections of articles of general interest',
 '00B15': 'Collections of articles of miscellaneous specific interest',
 '00B20': 'Proceedings of conferences of general interest',
 '00B25': 'Proceedings of conferences of miscellaneous specific interest',
 '00B30': 'Festschriften',
 '00B50': 'Collect

In [184]:
## Good, but we can improve it by removing the '\\(' and '\\)' characters

for item in dict.items():
    k , v = item
    v = v.replace('\\(','')
    v = v.replace('\\)','')
    dict[k] = v

dict

{'00A05': 'Mathematics in general',
 '00A08': 'Recreational mathematics',
 '00A09': 'Popularization of mathematics',
 '00A15': 'Bibliographies for mathematics in general',
 '00A17': 'External book reviews',
 '00A20': 'Dictionaries and other general reference works',
 '00A22': 'Formularies',
 '00A27': 'Lists of open problems',
 '00A30': 'Philosophy of mathematics',
 '00A64': 'Mathematics and literature',
 '00A65': 'Mathematics and music',
 '00A66': 'Mathematics and visual arts',
 '00A67': 'Mathematics and architecture',
 '00A71': 'General theory of mathematical modeling',
 '00A72': 'General theory of simulation',
 '00A79': 'Physics',
 '00B05': 'Collections of abstracts of lectures',
 '00B10': 'Collections of articles of general interest',
 '00B15': 'Collections of articles of miscellaneous specific interest',
 '00B20': 'Proceedings of conferences of general interest',
 '00B25': 'Proceedings of conferences of miscellaneous specific interest',
 '00B30': 'Festschriften',
 '00B50': 'Collect

In [185]:
!pip install unidecode

                                              0.0/235.9 kB ? eta -:--:--
     -------------------------------------  235.5/235.9 kB 7.3 MB/s eta 0:00:01
     -------------------------------------  235.5/235.9 kB 7.3 MB/s eta 0:00:01
     -------------------------------------- 235.9/235.9 kB 2.4 MB/s eta 0:00:00


In [202]:
from unidecode import unidecode

dict_stripped_accents = {}

for item in dict.items():
    k , v = item
    dict_stripped_accents[k] = unidecode(v)


In [207]:
with open('./data/msc.json','w') as file:
    json = json.dumps(dict_stripped_accents)
    file.write(json)

In [208]:
importlib.reload(util)

util.msc_tags()

{'00A05': 'Mathematics in general',
 '00A08': 'Recreational mathematics',
 '00A09': 'Popularization of mathematics',
 '00A15': 'Bibliographies for mathematics in general',
 '00A17': 'External book reviews',
 '00A20': 'Dictionaries and other general reference works',
 '00A22': 'Formularies',
 '00A27': 'Lists of open problems',
 '00A30': 'Philosophy of mathematics',
 '00A64': 'Mathematics and literature',
 '00A65': 'Mathematics and music',
 '00A66': 'Mathematics and visual arts',
 '00A67': 'Mathematics and architecture',
 '00A71': 'General theory of mathematical modeling',
 '00A72': 'General theory of simulation',
 '00A79': 'Physics',
 '00B05': 'Collections of abstracts of lectures',
 '00B10': 'Collections of articles of general interest',
 '00B15': 'Collections of articles of miscellaneous specific interest',
 '00B20': 'Proceedings of conferences of general interest',
 '00B25': 'Proceedings of conferences of miscellaneous specific interest',
 '00B30': 'Festschriften',
 '00B50': 'Collect

In [209]:
from library_class import Library

lib = Library()

lib.load_from_query(query_string='cat:math.AP',max_results=5000)

In [210]:
lib.raw_lib

,entry_id,updated,published,title,summary,comment,journal_ref,doi,primary_category,categories,pdf_url,authors,links
0,http://arxiv.org/abs/2306.03752v2,2023-06-15 17:15:27+00:00,2023-06-06 15:10:26+00:00,On the inviscid limit connecting Brinkman's and Darcy's models of tissue growth with nonlinear pressure,"Several recent papers have addressed modelling of the tissue growth by the\nmulti-phase models where the velocity is related to the pressure by one of the\nphysical laws (Stoke's, Brinkman's or Darcy's). While each of these models has\nbeen extensively studied, not so much is known about the connection between\nthem. In the recent paper (arXiv:2303.10620), assuming the linear form of the\npressure, the Authors connected two multi-phase models by an inviscid limit:\nthe viscoelastic one (of Brinkman's type) and the inviscid one (of Darcy's\ntype). Here, we prove that the same is true for a nonlinear, power-law\npressure. The new ingredient is that we use relation between the pressure $p$\nand the Brinkman potential $W$ to deduce compactness in space of $p$ from the\ncompactness in space of $W$.",13 pages + appendix + bibliography; in the new version a missing\n reference in the Appendix was added,None,None,math.AP,"[math.AP, 35K45, 35K65, 35J60, 35Q92, 92C10]",http://arxiv.org/pdf/2306.03752v2,"[Charles Elbar, Jakub Skrzeczkowski]","[http://arxiv.org/abs/2306.03752v2, http://arxiv.org/pdf/2306.03752v2]"
1,http://arxiv.org/abs/2306.09213v1,2023-06-15 15:50:01+00:00,2023-06-15 15:50:01+00:00,Stationarity and Fredholm theory in subextremal Kerr-de Sitter spacetimes,"In a recent paper, we proved that solutions to linear wave equations in a\nsubextremal Kerr-de Sitter spacetime have asymptotic expansions in quasinormal\nmodes up to a decay order given by the normally hyperbolic trapping, extending\nthe existing results. One central ingredient in the argument was a new\ndefinition of quasinormal modes, where a non-standard choice of stationary\nKilling vector field had to be used in order for the Fredholm theory to be\napplicable. In this paper, we show that there is in fact a variety of allowed\nchoices of stationary Killing vector fields. In particular, the horizon Killing\nvector fields work for the analysis, in which case one of the corresponding\nergoregions is completely removed.",13 pages. arXiv admin note: text overlap with arXiv:2112.01355,None,None,math.AP,"[math.AP, gr-qc, math.DG, 35L05, 35P25, 58J45, 83C30]",http://arxiv.org/pdf/2306.09213v1,"[Oliver Petersen, András Vasy]","[http://arxiv.org/abs/2306.09213v1, http://arxiv.org/pdf/2306.09213v1]"
2,http://arxiv.org/abs/2305.14134v2,2023-06-15 14:55:58+00:00,2023-05-23 14:57:51+00:00,"Remarks on paper ""Two-term spectral asymptotics in linear elasticity''","In this note, by pointing out several serious mistakes in \cite{CaFrLeVa-23}\nwe show that the conclusions published by Matteo Capoferri, Leonid Friedlander,\nMichael Levitin and Dmitri Vassiliev (J Geom Anal (2023)33:242) are completely\nwrong. Then, we explain the correctness of proof of Theorem 1.1 in our paper\n\cite{Liu-21} by giving some remarks and putting the whole proof in Appendix\n(see also \cite{Liu-22b} and \cite{Liu-22c}).",30 pages,None,None,math.SP,"[math.SP, math-ph, math.AP, math.DG, math.MP]",http://arxiv.org/pdf/2305.14134v2,[Genqian Liu],"[http://arxiv.org/abs/2305.14134v2, http://arxiv.org/pdf/2305.14134v2]"
3,http://arxiv.org/abs/2306.09168v1,2023-06-15 14:45:57+00:00,2023-06-15 14:45:57+00:00,Existence and uniqueness of weak solutions to the Smoluchowski coagulation equation with source and sedimentation,"This article is devoted to a generalized version of Smoluchowski's\ncoagulation equation. This model describes the time evolution of a system of\naggregating particles under the effect of external input and output particles.\nWe show that for a large class of coagulation kernels, output rates, and\nexponentially decaying input rates, there is a weak solution. Moreover, the\nsolution satisfies

In [218]:
import importlib
importlib.reload(util)

lib.clean_library()

In [220]:
lib.clean_lib.sample(10)

,title,summary,authors,primary_category,categories,hyph_in_summary,hyph_in_title,msc_tags
4996,Long-time asymptotic estimate and a related inverse source problem for time-fractional wave equations,"Lying between traditional parabolic and hyperbolic equations, time-fractional wave equations of order LATEX in time inherit both decaying and oscillating properties. In this article, we establish a long-time asymptotic estimate for homogeneous time-fractional wave equations, which readily implies the strict positivity/negativity of the solution for LATEX under some sign conditions on initial values. As a direct application, we prove the uniqueness for a related inverse source problem on determining the temporal component.","[Xinchi Huang, Yikan Liu]",math.AP,[math.AP],"[long-time, time-fractional]","[time-fractional, Long-time]",None
1412,Convergence rates in the nonrelativistic limit of the cubic Klein-Gordon equation,"In this paper, we study the nonrelativistic limit of the cubic nonlinear Klein-Gordon equation in LATEX with a small parameter LATEX which is inversely proportional to the speed of light. We show that the cubic nonlinear Klein-Gordon equation converges to the cubic nonlinear Schrodinger equation with a convergence rate of order LATEX In particular, for the defocusing case and smooth initial data, we prove error estimates of the form LATEX at time LATEX which is valid up to long time of order LATEX while for nonsmooth initial data, we prove error estimates of the form LATEX at time LATEX which is valid up to long time of order LATEX These specific forms of error estimates coincide with the numerical results obtained in .","[Weizhu Bao, Yong Lu, Zhifei Zhang]",math.AP,[math.AP],[Klein-Gordon],[Klein-Gordon],None
3812,On potential theory of Markov processes with jump kernels decaying at the boundary,"Motivated by some recent potential theoretic results on subordinate killed Levy processes in open subsets of the Euclidean space, we study processes in an open set LATEX defined via Dirichlet forms with jump kernels of the form LATEX and critical killing functions. Here LATEX is the Levy density of an isotropic stable process (or more generally, a pure jump isotropic unimodal Levy process) in LATEX The main novelty is that the term LATEX tends to 0 when LATEX or LATEX approach the boundary of LATEX Under some general assumptions on LATEX we construct the corresponding process and prove that non-negative harmonic functions of the process satisfy the Harnack inequality and Carleson's estimate. We give several examples of boundary terms satisfying those assumptions. The examples depend on four parameters, LATEX LATEX roughly governing the decay of the boundary term near the boundary of LATEX In the second part of this paper, we specialise to the case of the half-space LATEX the LATEX kernel LATEX and the killing function$appa(x)=c x_d^{-\alpha}$, LATEX where LATEX is a positive constant. Our main result in this part is a boundary Harnack principle which says that, for any LATEX there are values of the parameters LATEX LATEX and the constant LATEX such that non-negative harmonic functions of the process must decay at the rate LATEX if they vanish near a portion of the boundary. We further show that there are values of the parameters LATEX LATEX for which the boundary Harnack principle fails despite the fact that Carleson's estimate is valid.","[Panki Kim, Renming Song, Zoran Vondraček]",math.PR,"[math.PR, math.AP, math.FA, 60J45, 60J50, 60J75]","[half-space, non-negative]",None,[Boundary theory for Markov processes]
3040,Fractional Zernike functions,"We consider and provide an accurate study for the fractional Zernike functions on the punctured unit disc, generalizing the classical Zernike polynomials and their associated LATEX Zernike functions. Mainly, we give the spectral realization of the latter ones and show that they are orthogonal LATEX for certain perturbed magnetic (hyperbolic) Laplacian. The algebraic and analytic pr

## 06.17.23

In [9]:
## We are going to pull a library of the most recent 20k articles from pde and spectral theory.

import util
import pandas as pd
import numpy as np
from library_class import Library

lib = Library()
lib.load_from_query(query_string='cat:math.AP OR math.SP',max_results=2e4)




In [11]:
raw_lib = lib.raw_lib
raw_lib.to_parquet('./data/APSP.parquet')

In [12]:
## Is the list information preserved?

df = pd.read_parquet('./data/APSP.parquet')

In [20]:
lib.clean_library()
lib.clean_lib.head()

,title,summary,authors,primary_category,categories,hyph_in_summary,hyph_in_title,msc_tags
0,The Laplace spectrum on conformally compact ma...,We consider the spectrum of the Laplace operat...,"[Nelia Charalambous, Julie Rowlett]",math.SP,"[math.SP, 58c40]",None,None,None
1,On the inviscid limit connecting Brinkman's an...,Several recent papers have addressed modelling...,"[Charles Elbar, Jakub Skrzeczkowski]",math.AP,"[math.AP, 35K45, 35K65, 35J60, 35Q92, 92C10]","[power-law, multi-phase]",None,[Initial value problems for second-order parab...
2,A sparse approximation of the Lieb functional ...,The aim of this paper is to present new sparsi...,"[Virginie Ehrlacher, Luca Nenna]",math-ph,"[math-ph, math.MP, math.OC, math.SP]","[so-called, anti-symmetric, Levy-Lieb, trace-c...",None,None
3,Stationarity and Fredholm theory in subextrema...,"In a recent paper, we proved that solutions to...","[Oliver Petersen, András Vasy]",math.AP,"[math.AP, gr-qc, math.DG, 35L05, 35P25, 58J45,...","[non-standard, Kerr-de]",[Kerr-de],[Wave equation]
4,"Remarks on paper ""Two-term spectral asymptotic...","In this note, by pointing out several serious ...",[Genqian Liu],math.SP,"[math.SP, math-ph, math.AP, math.DG, math.MP]",None,[Two-term],None


In [21]:
pd.set_option('display.max_colwidth', 0)

In [3]:
from cleaning import cleaning
import pandas as pd
import importlib
importlib.reload(cleaning)

data = pd.read_parquet('./data/APSP.parquet')

clean_data = cleaning.main(
    raw_arxiv_results=data,path_to_embeddings='./data/APSP_mini_vec.parquet'
)


In [4]:
pd.set_option('display.max_colwidth', 0)
clean_data.head()

,clean_title,clean_abstract,authors,msc_tags,msc_cos_sim
0,The Laplace spectrum on conformally compact manifolds,"We consider the spectrum of the Laplace operator acting on LATEX over a conformally compact manifold for LATEX We prove that for LATEX this spectrum always contains an open region of the complex plane. We further show that the spectrum is contained within a certain parabolic region of the complex plane. These regions depend on the value of LATEX the dimension of the manifold, and the values of the sectional curvatures approaching the boundary.","[Nelia Charalambous, Julie Rowlett]",None,NaN
1,On the inviscid limit connecting Brinkman's and Darcy's models of tissue growth with nonlinear pressure,"Several recent papers have addressed modelling of the tissue growth by the multi-phase models where the velocity is related to the pressure by one of the physical laws (Stoke's, Brinkman's or Darcy's). While each of these models has been extensively studied, not so much is known about the connection between them. In the recent paper (arXiv:2303.10620), assuming the linear form of the pressure, the Authors connected two multi-phase models by an inviscid limit: the viscoelastic one (of Brinkman's type) and the inviscid one (of Darcy's type). Here, we prove that the same is true for a nonlinear, power-law pressure. The new ingredient is that we use relation between the pressure LATEX and the Brinkman potential LATEX to deduce compactness in space of LATEX from the compactness in space of LATEX","[Charles Elbar, Jakub Skrzeczkowski]","[Initial value problems for second-order parabolic systems, Degenerate parabolic equations, Nonlinear elliptic equations, Biomechanics]",NaN
2,A sparse approximation of the Lieb functional with moment constraints,"The aim of this paper is to present new sparsity results about the so-called Lieb functional, which is a key quantity in Density Functional Theory for electronic structure calculations for molecules. The Lieb functional was actually shown by Lieb to be a convexification of the so-called Levy-Lieb functional. Given an electronic density for a system of LATEX electrons, which may be seen as a probability density on LATEX the value of the Lieb functional for this density is defined as the solution of a quantum multi-marginal optimal transport problem, which reads as a minimization problem defined on the set of trace-class operators acting on the space of electronic wave-functions that are anti-symmetric LATEX functions of LATEX with partial trace equal to the prescribed electronic density. We introduce a relaxation of this quantum optimal transport problem where the full partial trace constraint is replaced by a finite number of moment constraints on the partial trace of the set of operators. We show that, under mild assumptions on the electronic density, there exist sparse minimizers to the resulting moment constrained approximation of the Lieb (MCAL) functional that read as operators with rank at most equal to the number of moment constraints. We also prove under appropriate assumptions on the set of moment functions that the value of the MCAL functional converges to the value of the exact Lieb functional as the number of moments go to infinity. We also prove some rates of convergence on the associated approximation of the ground state energy. We finally study the mathematical properties of the associated dual problem.","[Virginie Ehrlacher, Luca Nenna]",None,NaN
3,Stationarity and Fredholm theory in subextremal Kerr-de Sitter spacetimes,"In a recent paper, we proved that solutions to linear wave equations in a subextremal Kerr-de Sitter spacetime have asymptotic expansions in quasinormal modes up to a decay order given by the normally hyperbolic trapping, extending the existing results. One central ingredient in the argument was a new definition of quasinormal modes, where a non-standard choice of stationary Killing vector field had to be used in order for the Fredholm theory to be appli